# Damage Calculation 

This Notebook shows a general calculation stream for a nominal damage calculation.

1. Define load cycles
2. Define the material parameter
3. Select the damage calculation method (Miner elementary, Miner-Haibach, ...)
4. Calculate the damage for every load level and the damage sum
5. Calculate the failure probability with or w/o field scatter

In [30]:
import numpy as np
import pandas as pd
import pylife.strength.fatigue
import matplotlib.pyplot as plt
import plotly.express as px

### 1. Define load cycles

In [52]:
import pickle
loads = pickle.load(open("collectives.p", "rb"))
loads
load_cycles =  pd.DataFrame(
    {"amplitude": [100, 50, 75, 25],
     "cycles": [1e3, 5e3, 10e3, 25e3],
     }
    )

In [53]:
load_cycles

,amplitude,cycles
0,100,1000.0
1,50,5000.0
2,75,10000.0
3,25,25000.0


## 2. Define the material parameter

In [54]:
k_1 = 8
mat = pd.Series({
    'k_1': k_1,
    'k_2' : 2 * k_1 - 1,
    'ND': 1.0e6,
    'SD': 100.0,
    'TN': 12.,
    'TS': 1.1
})
display(mat.to_frame().T)

,k_1,k_2,ND,SD,TN,TS
0,8.0,15.0,1000000.0,100.0,12.0,1.1


### Damage Calculation
Now we can calculate the damage for every loadstep and summarize this damage to get the total damage.

In [55]:
damage = {
    "miner_original": mat.fatigue.damage(load_cycles),
    "miner_elementary": mat.fatigue.miner_elementary().damage(load_cycles),
    "miner_haibach": mat.fatigue.miner_haibach().damage(load_cycles),
}
pr_out = [print("total damage sum " + k + " :  %.2e" %v.sum()) for k, v in damage.items()]

total damage sum miner_original :  1.13e-03
total damage sum miner_elementary :  2.02e-03
total damage sum miner_haibach :  1.13e-03


#### Plot the damage vs collectives

define new materials and plot it

In [58]:
wc = mat.woehler
cyc = np.logspace(1, 12, 200)
sn_df = pd.DataFrame({"cycles": cyc,
                     "amplitude": wc.basquin_load(cyc)})
fig = px.bar(load_cycles, x="cycles", y="amplitude", orientation='h',   log_x=True, log_y=True)
fig.add_scatter(x=sn_df.cycles, y=sn_df.amplitude, name="sn curve")
fig.show()